In [ ]:
import numpy as np
from PIL import Image
import tensorflow as tf

In [ ]:
def road_prediction(model_path, sat_path):
  model = tf.keras.models.load_model(model_path,
                                     custom_objects = None, 
                                     compile = False, 
                                     options = None
                                     )
 
  test = np.zeros((1, 256, 256, 3)).astype('float')
 
  img = Image.open(sat_path)
  img = img.resize((256, 256))
  img = np.array(img)/255.
  test[0] = img
 
  predicted = model.predict(test)
 
  road = np.squeeze(predicted, axis = 0)
  road = np.squeeze(road, axis = -1)
  road = Image.fromarray((road * 255).astype(np.uint8))

  return road

In [ ]:
def building_no_alpha(building_path):
  img = Image.open(building_path)
  img = img.convert("RGBA")

  datas = img.getdata()
  newData = []

  for item in datas:
    if item[0] == 255 and item[1] == 255 and item[2] == 255:
      newData.append((255, 255, 255, 0))
    else:
      newData.append(item)

  img.putdata(newData)

  return img

In [ ]:
def map_overlay(model_path, building_path, sat_path):
  background = road_prediction(model_path, sat_path)
  background = background.convert('RGBA')
  background = background.resize((512,512))
  foreground = building_no_alpha(building_path)
  foreground = foreground.resize((512,512))

  background.paste(foreground, (0, 0), foreground)

  return background

In [ ]:
def save(model_path, pre_building_path, pre_sat_path, post_building_path, post_sat_path, save_path):
  pre_id = pre_sat_path.split('_')[-3]
  pre_map = map_overlay(model_path, pre_building_path, pre_sat_path)
  pre_map.save(save_path + f'/pre_map_{pre_id}.png', 'PNG')

  post_id = post_sat_path.split('_')[-3]
  post_map = map_overlay(model_path, post_building_path, post_sat_path)
  post_map.save(save_path + f'/post_map_{post_id}.png', 'PNG')

In [ ]:
model_path = '/content/drive/MyDrive/Bangkit Capstone Project/Road Extraction/main/models/road_extraction_resunet.hdf5'
pre_building_path = '/content/drive/MyDrive/Bangkit Capstone Project/Road Extraction/main/images/02 building masks/original_mask 02.png'
pre_sat_path = '/content/drive/MyDrive/Bangkit Capstone Project/Road Extraction/main/images/01 satellite images/palu-tsunami_00000002_pre_disaster.png'
post_building_path = '/content/drive/MyDrive/Bangkit Capstone Project/Road Extraction/main/images/02 building masks/predicted_mask 02.png'
post_sat_path = '/content/drive/MyDrive/Bangkit Capstone Project/Road Extraction/main/images/01 satellite images/palu-tsunami_00000002_post_disaster.png'
save_path = '/content/drive/MyDrive/Bangkit Capstone Project/Road Extraction/main/images/05 road and building masks'

In [ ]:
save(model_path, pre_building_path, pre_sat_path, post_building_path, post_sat_path, save_path)